In [1]:
import requests
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryType
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from typing import List
from llama_index.core.schema import MetadataMode
import json

In [2]:
# 분석할 PDF 파일을 웹에서 다운로드.
url = "https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf"
filename = "2023_북한인권보고서.pdf"

response = requests.get(url)
with open(filename, "wb") as f:
    f.write(response.content)

print(f"{filename} 다운로드 완료")

2023_북한인권보고서.pdf 다운로드 완료


In [3]:
# 라마인덱스의 핵심 설정: LLM, 임베딩 모델, 문서 분할 방식을 전역으로 설정
# GPT-4.1를 언어 모델로 사용
Settings.llm = OpenAI(model="gpt-4.1", temperature=0.2)
# 임베딩 모델 사용
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")
Settings.chunk_size = 300 # 문서를 300자 단위로 분할
Settings.chunk_overlap = 100 # 문맥 유지를 위해 청크 간 100자 중복

# PDF 문서를 읽고 벡터 인덱스 생성
# PDF 문서 로더
reader = SimpleDirectoryReader(input_files=["2023_북한인권보고서.pdf"])
# 문서에서 텍스트 추출
documents = reader.load_data()
# 추출된 텍스트로 벡터 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

In [4]:
class DocumentScorer(BaseNodePostprocessor):
    # LLM을 사용해 문서의 관련성을 정밀하게 평가하고 점수를 매기는 클래스
    def evaluate_document(self, query: str, content: str) -> float:
        # LLM을 사용해 문서와 쿼리 간의 의미적 관련성을 1-10점으로 평가
        prompt = f"""
        아래 주어진 질문과 문서의 관련성을 평가해주세요.

        [평가 기준]
        - 문서가 질문에서 요구하는 정보를 직접적으로 포함하면 8-10점
        - 문서가 질문과 관련된 맥락을 포함하지만 직접적인 답이 아니면 4-7점
        - 문서가 질문과 거의 관련이 없으면 1-3점

        [주의사항]
        - 단순히 비슷한 단어가 등장하는 것은 높은 점수의 근거가 될 수 없습니다
        - 질문의 의도와 문맥을 정확히 파악하여 평가해주세요
        - 시간, 장소, 수치 등 구체적인 정보의 일치 여부를 중요하게 고려해주세요

        질문: {query}
        문서: {content}

        응답은 반드시 다음 JSON 형식이어야 합니다:
        {{"relevance_score": float}}
        """

        try:
            # LLM에 프롬프트를 전송하고 JSON 형식의 응답을 받음
            response = Settings.llm.complete(prompt)
            # 응답에서 relevance_score 값을 추출
            score = json.loads(response.text)["relevance_score"]
            # 점수를 float로 변환하여 반환
            return float(score)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return 5.0 # 에러 발생시 중간 점수로 처리하여 시스템 안정성 유지
    
    def _postprocess_nodes(self, nodes: List[NodeWithScore], query: QueryType) -> List[NodeWithScore]:
        # 벡터 검색으로 찾은 4개 문서를 LLM으로 재평가하여 최적의 2개 선택
        print('\n=== LLM이 4개의 검색 결과에 대해서 관련성을 평가합니다. ===')
        scored_docs = []
        for node in nodes:
            # 현재 처리 중인 문서 노드에서 순수 텍스트 컨텐츠만 추출
            content = node.node.get_content(metadata_mode=MetadataMode.NONE)
            # LLM으로 문서 관련성 점수 계산 (1-10 사이 점수)
            score = self.evaluate_document(str(query), content)
            # 디버깅/모니터링을 위해 각 문서의 내용과 점수를 출력
            print(f"\nLLM 기반의 평가:\n{content}\n=> 점수: {score}\n")
            # 현재 노드와 계산된 점수를 튜플로 저장
            scored_docs.append((node, score))

        # 모든 문서를 점수 기준 내림차순으로 정렬하고 상위 2개만 선택하여 반환
        ranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
        return [node for node, _ in ranked_docs[:2]]

In [5]:
class SemanticRanker(BaseRetriever):
    # 벡터 검색 결과에 LLM 기반 의미적 평가를 적용하여 최적의 문서를 선별하는 시스템

    def __init__(self, index, scorer):
        # 생성자에서 벡터 검색용 인덱스와 LLM 기반 문서 평가기 인스턴스를 받아 저장
        self.index = index # 벡터 검색용 인덱스
        self.scorer = scorer # LLM 기반 문서 평가기
    
    def _retrieve(self, query: str) -> List[NodeWithScore]:
        # 벡터 검색으로 유사도 기반 후보 문서 4개를 추출하고 LLM으로 재평가
        vector_results = self.index.as_retriever(similarity_top_k=4).retrieve(query)
    
        # 초기 벡터 검색 결과를 디버깅/분석용으로 출력
        print("\n=== 실제 검색 결과 (Top 4) ===")
        for i, node in enumerate(vector_results, 1):
            print(f"\n검색 문서 {i}:")
            print(node.node.get_content(metadata_mode=MetadataMode.NONE))
    
        # LLM으로 문서들을 재평가하고 재정렬하여 최적의 2개 선택
        reranked_results = self.scorer._postprocess_nodes(vector_results, query)
    
        # 최종 선별된 문서를 디버깅/분석용으로 출력
        print("\n=== LLM의 리랭킹 결과 (Top 2) ===")
        for i, node in enumerate(reranked_results, 1):
            print(f"\n검색 문서 {i}:")
            print(node.node.get_content(metadata_mode=MetadataMode.NONE))
        
        return reranked_results

In [6]:
# 문서 평가 및 검색 시스템 선언(초기화)
scorer = DocumentScorer() # LLM 기반 문서 평가기 생성
ranker = SemanticRanker(index, scorer) # 벡터 검색과 LLM 평가를 결합한 시스템 생성
query_engine = RetrieverQueryEngine(retriever=ranker) # 최종 질의응답 엔진 생성

# 실제 쿼리 실행
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"
print(f"\n질문: {query}")
response = query_engine.query(query) # 쿼리 실행하여 응답 생성
print(f"\n최종 답: {response}") # 최종 응답 출력


질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식

=== 실제 검색 결과 (Top 4) ===

검색 문서 1:
대체로 합영·합작회사, 
외화벌이 기관 등 운영이 잘되는 경우였으며, 보수를 달러나 위안
화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평양
의 외화벌이 사업소에서는 보수 50달러를 월 2회로 나누어 현금으
로 지급하였다고 하는 사례가 있었고, 평양 외화벌이 식당에서는 매

검색 문서 2:
2023 북한인권보고서
252
며, 배급량의 80%는 강냉이로 쌀은 명절에만 배급되었다는 진술이 
있었다. 기업소에서 배급표는 매월 두 차례(7~8일경 및 21~22일경 
상·하순) 지급되었고, 거주지 배급소에서 식량으로 바꾸면 되었다고 
한다. 
식량배급이 되더라도 규정에 미치지 못하는 매우 적은 양을 받았
던 경우도 많았다.

검색 문서 3:
외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았
다는 증언이 있었다.

검색 문서 4:
279 체제보위기관 등을 
제외한 기관·기업소에서는 식량자력조달을 강화하여 노동자에 대
한 식량배급은 기관·기업소에서 독자적으로 이뤄지고 있는 것으로 
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다.

=== LLM이 4개의 검색 결과에 대해서 관련성을 평가합니다. ===

LLM 기반의 평가:
대체로 합영·합작회사, 
외화벌이 기관 등 운영이 잘되는 경우였으며, 보수를 달러나 위안
화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평

In [7]:
import urllib.request
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.postprocessor import SentenceTransformerRerank

In [10]:
# 분석할 PDF 파일을 웹에서 다운로드.
urllib.request.urlretrieve("https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")

('2023_북한인권보고서.pdf', <http.client.HTTPMessage at 0x337937fd0>)

In [11]:
# LlamaIndex의 핵심 설정: LLM, 임베딩 모델, 문서 분할 방식을 전역으로 설정
# GPT-4.1을 언어 모델로 사용
Settings.llm = OpenAI(model="gpt-4.1", temperature=0.2)
# 임베딩 모델 사용
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")
Settings.chunk_size = 300 # 문서를 300자 단위로 분할
Settings.chunk_overlap = 100 # 문맥 유지를 위해 청크 간 100자 중복

# PDF 문서를 읽고 벡터 인덱스 생성
# PDF 문서 로더
reader = SimpleDirectoryReader(input_files=["2023_북한인권보고서.pdf"])
# 문서에서 텍스트 추출
documents = reader.load_data()
# 추출된 텍스트로 벡터 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

In [12]:
# 기본 검색 엔진 (리랭킹 없음)
basic_query_engine = index.as_query_engine(
    similarity_top_k=4
)

# Reranker 설정
reranker = SentenceTransformerRerank(
    model="BAAI/bge-reranker-v2-m3",
    top_n=2
)

# 리랭킹이 포함된 검색 엔진
rerank_query_engine = index.as_query_engine(
    similarity_top_k=4,
    node_postprocessors=[reranker]
)

/Users/gaheeyoon/llamaindex/ch07/ch07_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# 쿼리 실행
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"

print("=== 기본 검색 엔진 검색 결과 ===")
basic_response = basic_query_engine.query(query)
print(f"\n질문: {query}")
print(f"답변: {basic_response.response}")
print("\n검색된 문서:")
for i, node in enumerate(basic_response.source_nodes):
    print(f"\n검색 문서 {i+1}:")
    print(node.node.get_content())
    print("---")

=== 기본 검색 엔진 검색 결과 ===

질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
답변: 2019년 평양시에서 기업소 운전원으로 일한 노동자는 매월 쌀, 설탕, 기름, 야채, 돼지고기 등을 배급받아 식량이 부족하지 않았다고 하였으며, 중앙당 산하의 기업소에서는 매월 쌀 6kg 정도, 기름 5ℓ, 설탕 2kg, 맛내기 2봉지, 돼지고기 2kg, 닭고기 1마리 정도를 배급받았다는 증언이 있다.

검색된 문서:

검색 문서 1:
대체로 합영·합작회사, 
외화벌이 기관 등 운영이 잘되는 경우였으며, 보수를 달러나 위안
화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평양
의 외화벌이 사업소에서는 보수 50달러를 월 2회로 나누어 현금으
로 지급하였다고 하는 사례가 있었고, 평양 외화벌이 식당에서는 매
---

검색 문서 2:
2023 북한인권보고서
252
며, 배급량의 80%는 강냉이로 쌀은 명절에만 배급되었다는 진술이 
있었다. 기업소에서 배급표는 매월 두 차례(7~8일경 및 21~22일경 
상·하순) 지급되었고, 거주지 배급소에서 식량으로 바꾸면 되었다고 
한다. 
식량배급이 되더라도 규정에 미치지 못하는 매우 적은 양을 받았
던 경우도 많았다.
---

검색 문서 3:
외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았
다는 증언이 있었다.
---

검색 문서 4:
2017년 
평양시 소재 기업소는 노동자에게 한번 1개월분의 식량을 배급하였
으며, 1인당 2㎏ 정도만 지급되었다는 진술이 있었다. 2018년 양강
도 철도노동자에게 직장노동자 700g으로 규정대로 배급표가 나왔
지만, 실제 식량배급은 1년 동안 감자 15

In [14]:
print("\n\n=== 리랭킹 후 검색 결과 ===")
rerank_response = rerank_query_engine.query(query)
print(f"\n질문: {query}")
print(f"답변: {rerank_response.response}")
print("\n검색된 문서:")
for i, node in enumerate(rerank_response.source_nodes):
    print(f"\n검색 문서 {i+1}:")
    print(node.node.get_content())
    print("---")



=== 리랭킹 후 검색 결과 ===

질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
답변: 2019년 평양시 소재 기업소에서는 매월 쌀 6kg 정도, 기름 5ℓ, 설탕 2kg, 맛내기 2봉지, 돼지고기 2kg, 닭고기 1마리 정도를 배급받았다.

검색된 문서:

검색 문서 1:
외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았
다는 증언이 있었다.
---

검색 문서 2:
2017년 
평양시 소재 기업소는 노동자에게 한번 1개월분의 식량을 배급하였
으며, 1인당 2㎏ 정도만 지급되었다는 진술이 있었다. 2018년 양강
도 철도노동자에게 직장노동자 700g으로 규정대로 배급표가 나왔
지만, 실제 식량배급은 1년 동안 감자 150㎏을 한번 받았다는 사례
와 2019년 시인민위원회 기관에서 일하던 가족이 1년에 한번 감자 
200kg을 배급받았지만, 운송비용 등으로 2만 원을 내야했다는 진
술도 있었다.
---


In [15]:
import requests
import openai
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.openai import OpenAIEmbedding

In [16]:
# 분석할 PDF 파일을 웹에서 다운로드.
url = "https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf"
filename = "2023_북한인권보고서.pdf"

response = requests.get(url)
with open(filename, "wb") as f:
    f.write(response.content)

print(f"{filename} 다운로드 완료")

2023_북한인권보고서.pdf 다운로드 완료


In [17]:
# 라마인덱스의 핵심 설정: LLM, 임베딩 모델, 문서 분할 방식을 전역으로 설정
# GPT-4.1을 언어 모델로 사용
Settings.llm = OpenAI(model="gpt-4.1", temperature=0.2)
# 임베딩 모델 사용
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")
Settings.chunk_size = 300 # 문서를 300자 단위로 분할
Settings.chunk_overlap = 100 # 문맥 유지를 위해 청크 간 100자 중복
# PDF 문서를 읽고 벡터 인덱스 생성
# PDF 문서 로더
reader = SimpleDirectoryReader(input_files=["2023_북한인권보고서.pdf"])
# 문서에서 텍스트 추출
documents = reader.load_data()
# 추출된 텍스트로 벡터 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

In [18]:
# 기본 검색기 설정
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=4
)

In [19]:
def generate_hypothetical_doc(question: str) -> str:
    """질문에 대한 가상의 이상적인 답변 문서 생성"""
    prompt = f"""주어진 질문에 대해, 마치 실제 문서에서 발췌한 것 같은 이상적인 답변을 작성해주세요. 단, 구체적인 수치, 날짜, 트렌드와 같은 상세 정보를 포함해야 합니다.

    질문: {question}
    답변:"""

    response = Settings.llm.complete(prompt)
    return response.text

In [20]:
def search_with_hyde(hypothetical_doc: str):
    """가상 문서를 이용해 실제 문서 검색"""
    nodes = retriever.retrieve(hypothetical_doc)
    return [
        {
            'content': node.node.get_content(),
            'score': node.score
        } for node in nodes
    ]

In [21]:
def generate_final_answer(question: str, relevant_docs: list) -> str:
    """검색된 문서를 바탕으로 최종 답변 생성"""
    context = "\n\n".join([doc['content'] for doc in relevant_docs])

    prompt = f"""다음 검색 결과를 바탕으로 질문에 답변해주세요. 검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

    검색 결과:
    {context}

    질문: {question}

    답변:"""
    
    response = Settings.llm.complete(prompt)
    return response.text

In [22]:
def process_query(question: str):
    """전체 HyDE 프로세스"""
    # 1. 가상 문서 생성
    print("1. 가상 문서 생성 중...")
    hypothetical_doc = generate_hypothetical_doc(question)
    print("\n가상 문서:", hypothetical_doc)

    # 2. 가상 문서로 검색
    print("\n2. 실제 문서 검색 중...")
    relevant_docs = search_with_hyde(hypothetical_doc)

    # 3. 최종 답변 생성
    print("\n3. 최종 답변 생성 중...")
    final_answer = generate_final_answer(question, relevant_docs)

    return {
        "hypothetical_doc": hypothetical_doc,
        "retrieved_docs": relevant_docs,
        "final_answer": final_answer
    }

In [23]:
question = "북한에서 강제로 이루어지는 조직 생활은 무엇이 있나요?"
result = process_query(question)

print("\n=== 프로세스 결과 ===")
print("\n[가상 문서]")
print(result["hypothetical_doc"])
print("\n[검색된 문서들]")
for idx, doc in enumerate(result["retrieved_docs"], 1):
    print(f"\n문서 {idx} (유사도 점수: {doc['score']:.4f}):")
    print(doc['content'])
print("\n[최종 답변]")
print(result["final_answer"])

1. 가상 문서 생성 중...

가상 문서: 북한에서 강제로 이루어지는 조직 생활에는 대표적으로 ‘생활총화’, ‘조직생활총화’, ‘당세포 생활’, ‘청년동맹 활동’, ‘여맹(여성동맹) 활동’ 등이 있습니다. 북한 주민들은 나이, 직업, 성별에 따라 각각의 조직에 소속되어 정기적으로 조직 생활에 참여해야 하며, 이는 법적·사회적 의무로 간주됩니다.

예를 들어, 모든 성인 남성은 ‘당세포’ 또는 ‘직장세포’에 소속되어 매주 1회 이상 ‘생활총화’에 참석해야 합니다. 이 자리에서는 한 주간의 사상·행동을 서로 비판하고, 당 정책 이행 상황을 점검합니다. 청소년과 청년들은 ‘사회주의애국청년동맹’(구 김일성사회주의청년동맹)에 소속되어 매주 1~2회 조직 생활을 하며, 여성들은 ‘조선민주여성동맹’(여맹) 활동에 참여합니다.

또한, 2020년대 들어 북한 당국은 조직 생활의 강도를 더욱 높여, 2023년 기준으로 ‘5대 교양실천강령’ 이행 점검, ‘충성의 80일 전투’ 등 각종 집단행동과 사상교육을 강화하고 있습니다. 조직 생활 불참 시에는 비판, 불이익, 심할 경우 처벌까지 받을 수 있어, 주민들은 사실상 자율적 선택권 없이 조직 생활에 참여하고 있습니다.

2. 실제 문서 검색 중...

3. 최종 답변 생성 중...

=== 프로세스 결과 ===

[가상 문서]
북한에서 강제로 이루어지는 조직 생활에는 대표적으로 ‘생활총화’, ‘조직생활총화’, ‘당세포 생활’, ‘청년동맹 활동’, ‘여맹(여성동맹) 활동’ 등이 있습니다. 북한 주민들은 나이, 직업, 성별에 따라 각각의 조직에 소속되어 정기적으로 조직 생활에 참여해야 하며, 이는 법적·사회적 의무로 간주됩니다.

예를 들어, 모든 성인 남성은 ‘당세포’ 또는 ‘직장세포’에 소속되어 매주 1회 이상 ‘생활총화’에 참석해야 합니다. 이 자리에서는 한 주간의 사상·행동을 서로 비판하고, 당 정책 이행 상황을 점검합니다. 청소년과 청년들은 ‘사회주의애국청년동맹’(구 김일성사회주의청년동맹)에 소속되어 매주 1~2회 조직 생